<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. We will be performing more interactive visual analytics using `Folium`.


## Objectives


Tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
#!pip install folium
!pip install wget
#!pip install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=8fe7ab5eadd5f3dda2ab1c6505625a0671645afed318548a7866f8fbfb76bf2b
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, we can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give we any intuitive insights about where are those launch sites. If we are very good at geography, we can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

folium.PolyLine(
    locations=[[0, -180], [0, 180]],  # From far left to far right on latitude 0
    color='red',
    weight=2,
    dash_array='5, 5'  # Dashed line for visibility
).add_to(site_map)

site_map

and we should find a small yellow circle near the city of Houston and we can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [8]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Loop through the DataFrame
for _, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    coordinate = [row['Lat'], row['Long']]

    # Add a circle with popup
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # Add a marker with label
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    ).add_to(site_map)

# Display the map (in Jupyter) or save as HTML
site_map

Now, we can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
  No, they are far from the Equator line.
- Are all launch sites in very close proximity to the coast?
  Yes, they are near from the coast lines.

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster().add_to(site_map)

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.head(10)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to the map (not strictly needed if added above, but harmless)
site_map.add_child(marker_cluster)

# Iterate over each launch record in the DataFrame
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    color = record['marker_color']

    folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


From the color-labeled markers in marker clusters, we should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


In [13]:
from folium.plugins import MousePosition

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topleft',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


Now zoom in to a launch site and explore its proximity to see if we can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


We can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56341  Lon: -80.56792
launch_site_lat = 28.56341
launch_site_lon = -80.57678
coastline_lat = 28.56355
coastline_lon = -80.56793
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.8647128165086165

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

midpoint = [(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2]
distance_marker = folium.Marker(
    location=midpoint,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)

_TODO:_ Similarly, We can draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find the their coordinates on the map first


In [18]:
launch_site_lat = 28.56341
launch_site_lon = -80.57678
railway_lat = 28.56336
railway_lon = -80.58695
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_railway

coordinates = [
    [launch_site_lat, launch_site_lon],
    [railway_lat, railway_lon]
]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

midpoint = [(launch_site_lat + railway_lat) / 2, (launch_site_lon + railway_lon) / 2]
distance_marker = folium.Marker(
    location=midpoint,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_railway)
    )
)
site_map.add_child(distance_marker)

In [19]:
launch_site_lat = 28.56341
launch_site_lon = -80.57678
highway_lat = 28.56351
highway_lon = -80.57081
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_highway

coordinates = [
    [launch_site_lat, launch_site_lon],
    [highway_lat, highway_lon]
]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

midpoint = [(launch_site_lat + highway_lat) / 2, (launch_site_lon + highway_lon) / 2]
distance_marker = folium.Marker(
    location=midpoint,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_highway)
    )
)
site_map.add_child(distance_marker)

In [20]:
launch_site_lat = 28.56341
launch_site_lon = -80.57678
city_lat = 28.07940
city_lon = -80.60780
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_city

coordinates = [
    [launch_site_lat, launch_site_lon],
    [city_lat, city_lon]
]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

midpoint = [(launch_site_lat + city_lat) / 2, (launch_site_lon + city_lon) / 2]
distance_marker = folium.Marker(
    location=midpoint,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_city)
    )
)
site_map.add_child(distance_marker)

After we plot distance lines to the proximities, we can answer the following questions easily:
- Are launch sites in close proximity to railways? Yes, 0.99km
- Are launch sites in close proximity to highways? Yes, 0.58km
- Are launch sites in close proximity to coastline? Yes, 0.86km
- Do launch sites keep certain distance away from cities? Yes, 53.92km

Also please try to explain your findings.


# Next Steps:

Now we have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, we will need to build a dashboard using Ploty Dash on detailed launch records.


Copyright © 2021 IBM Corporation. All rights reserved.
